In [1]:
import os
import torch
import zipfile
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
from accelerate import init_empty_weights
from datasets import Dataset
 
# Set environment variable to manage memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
 
# Specify directories and the path to the zip file
# save_dir = os.path.expanduser("llama_webtext/")
offload_dir = os.path.expanduser("llama_offload/")
# data_zip_path = "llama_webtext.7z"
# data_extract_path = os.path.expanduser("llama_webtext/data")
 
# Create directories if they don't exist
# os.makedirs(save_dir, exist_ok=True)
os.makedirs(offload_dir, exist_ok=True)
# os.makedirs(data_extract_path, exist_ok=True)
 
# Extract only the specified JSON file from the zip archive
target_file = "task024_cosmosqa_answer_generation.json"
# with zipfile.ZipFile(data_zip_path, 'r') as zip_ref:
#     zip_ref.extract(target_file, data_extract_path)
 
# Load tokenizer from Hugging Face
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
tokenizer.pad_token = tokenizer.eos_token
 
# Load the model with accelerate's offloading and device map auto-setup
with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(
        "meta-llama/Meta-Llama-3-8B",
        device_map="auto",
        offload_folder=offload_dir,
        load_in_8bit=True,
        llm_int8_enable_fp32_cpu_offload=True
    )
 
# Configure LoRA with reduced rank
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)
 
# Load and process the JSON data from the extracted file


C:\Users\panshul2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
C:\Users\panshul2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\accelerate\utils\modeling.py:841: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  _ = torch.tensor([0], device=i)
Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,

## Question Answering Fine Tuning Ahead:

In [2]:
target_file = r"task024_cosmosqa_answer_generation.json"
with open(target_file, 'r', encoding='utf-8-sig') as f:
    json_data = json.load(f)

# Extract text data (assuming a structure where the data you want is under 'Instances')

instances = json_data['Instances'][0:2223]
input_texts = [str(instance['input']) for instance in instances]  # Convert to string if not already
output_texts = [str(instance['output'][0]) if instance['output'] else "" for instance in instances]  # Handle missing output
print(input_texts, output_texts)


["Context: Good Old War and person L : I saw both of these bands Wednesday night , and they both blew me away . seriously . Good Old War is acoustic and makes me smile . I really can not help but be happy when I listen to them ; I think it 's the fact that they seemed so happy themselves when they played . \nQuestion: In the future , will this person go to see other bands play ?", "Context: I mean it : not one person said ANYTHING to me . They would have if I did something wrong , right ? But , this morning , I got a call from my guy at the temp agency and he said that I was no longer needed in that position , that I did n't need to go out there . \nQuestion: Why might have the temp agency tell me I am not needed at that position ?", 'Context: Leaving my shift Thursday day shift I arrived the same time as my partner just after six that evening and before long the radio erupted in dispatch tones . A car fleeing the police has crashed and landed on its roof with four separate people entr

In [3]:
 
# Convert the texts into a Hugging Face Dataset
ds = Dataset.from_dict({'input': input_texts, 'output': output_texts})

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["input"], examples["output"], truncation=True, padding="max_length", max_length=512)

# Apply tokenization and set format
tokenized_datasets = ds.map(tokenize_function, batched=True, remove_columns=["input", "output"])
tokenized_datasets.set_format("torch")

# Split dataset into train and eval
train_size = int(0.9 * len(tokenized_datasets))
train_dataset = tokenized_datasets.select(range(train_size))
eval_dataset = tokenized_datasets.select(range(train_size, len(tokenized_datasets)))

# Define data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Map: 100%|██████████| 2223/2223 [00:00<00:00, 5146.14 examples/s]


In [4]:
# Define training arguments
save_dir="finetuned-weights"
training_args = TrainingArguments(
    output_dir=save_dir,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_steps=500,
    save_steps=1000,
    save_total_limit=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    report_to="none",
    fp16=torch.cuda.is_available(),
)
 
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)
 
# Train the model
trainer.train()
 
# Evaluate the model
with torch.no_grad():
    eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")
 
# Save the model and tokenizer

model.save_pretrained(os.path.join(save_dir, "fine-tuned-llama-lora"))
tokenizer.save_pretrained(os.path.join(save_dir, "fine-tuned-llama-lora"))
 
# Test the model on a sample input
input_text = "Hello, what is the meaning of life?"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
 
with torch.no_grad():
    outputs = model.generate(inputs["input_ids"], max_length=50, num_return_sequences=1)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Generated output: {output_text}")
 
# Clear CUDA cache
torch.cuda.empty_cache()

C:\Users\panshul2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\panshul2\AppData\Local\Temp\ipykernel_142096\2425887148.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  0%|          | 0/1500 [00:00<?, ?it/s]C:\Users\panshul2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\models\llama\modeling_llama.py:602: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn

{'loss': 2.7404, 'grad_norm': 3.805873155593872, 'learning_rate': 3.3400000000000005e-05, 'epoch': 1.0}


                                                       
 33%|███▎      | 500/1500 [5:59:29<11:28:15, 41.30s/it]

{'eval_loss': 2.6262338161468506, 'eval_runtime': 1259.934, 'eval_samples_per_second': 0.177, 'eval_steps_per_second': 0.022, 'epoch': 1.0}


 67%|██████▋   | 1000/1500 [11:41:58<5:38:47, 40.66s/it] 

...

{'train_runtime': 65984.1098, 'train_samples_per_second': 0.091, 'train_steps_per_second': 0.023, 'train_loss': 2.543170084635417, 'epoch': 3.0}


100%|██████████| 28/28 [20:53<00:00, 44.75s/it]


Evaluation results: {'eval_loss': 2.516984701156616, 'eval_runtime': 1300.1216, 'eval_samples_per_second': 0.172, 'eval_steps_per_second': 0.022, 'epoch': 3.0}


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated output: Hello, what is the meaning of life? Is it to be happy, to make others happy, to be sad, to make others sad, to be in love, to make others in love, to be in hate, to make others in


## Question Generation Finetuning Ahead:

In [5]:
target_file = r"task074_squad1.1_question_generation.json"
with open(target_file, 'r', encoding='utf-8-sig') as f:
    json_data = json.load(f)

# Extract text data (assuming a structure where the data you want is under 'Instances')

instances = json_data['Instances'][0:2223]
input_texts = [str(instance['input']) for instance in instances]  # Convert to string if not already
output_texts = [str(instance['output'][0]) if instance['output'] else "" for instance in instances]  # Handle missing output
print(input_texts, output_texts)


['With the new millennium, Marvel Comics emerged from bankruptcy and again began diversifying its offerings. In 2001, Marvel withdrew from the Comics Code Authority and established its own Marvel Rating System for comics. The first title from this era to not have the code was X-Force #119 (October 2001). Marvel also created new imprints, such as MAX (an explicit-content line) and Marvel Adventures (developed for child audiences). In addition, the company created an alternate universe imprint, Ultimate Marvel, that allowed the company to reboot its major titles by revising and updating its characters to introduce to a new generation.', 'The UK national curriculum is adapted for local use. A range of qualifications are offered – from GCSE, A/S and A2, to Level 3 Diplomas and VRQ qualifications:', 'Greek Cypriots and Turkish Cypriots share a lot in common in their culture but also have differences. Several traditional food (such as souvla and halloumi) and beverages are similar, as well a

In [6]:
 
# Convert the texts into a Hugging Face Dataset
ds = Dataset.from_dict({'input': input_texts, 'output': output_texts})

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["input"], examples["output"], truncation=True, padding="max_length", max_length=512)

# Apply tokenization and set format
tokenized_datasets = ds.map(tokenize_function, batched=True, remove_columns=["input", "output"])
tokenized_datasets.set_format("torch")

# Split dataset into train and eval
train_size = int(0.9 * len(tokenized_datasets))
train_dataset = tokenized_datasets.select(range(train_size))
eval_dataset = tokenized_datasets.select(range(train_size, len(tokenized_datasets)))

# Define data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Map: 100%|██████████| 2223/2223 [00:00<00:00, 4939.62 examples/s]


In [ ]:
# Define training arguments
save_dir="finetuned-weights"
training_args = TrainingArguments(
    output_dir=save_dir,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_steps=500,
    save_steps=1000,
    save_total_limit=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    report_to="none",
    fp16=torch.cuda.is_available(),
)
 
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)
 
# Train the model
trainer.train()
 
# Evaluate the model
with torch.no_grad():
    eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")
 
# Save the model and tokenizer

model.save_pretrained(os.path.join(save_dir, "fine-tuned-llama-lora"))
tokenizer.save_pretrained(os.path.join(save_dir, "fine-tuned-llama-lora"))
 
# Test the model on a sample input
input_text = "Hello, what is the meaning of life?"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
 
with torch.no_grad():
    outputs = model.generate(inputs["input_ids"], max_length=50, num_return_sequences=1)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Generated output: {output_text}")
 
# Clear CUDA cache
torch.cuda.empty_cache()

C:\Users\panshul2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\panshul2\AppData\Local\Temp\ipykernel_142096\706721376.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
 33%|███▎      | 500/1500 [5:58:37<11:33:51, 41.63s/it]

{'loss': 1.8981, 'grad_norm': 2.8968536853790283, 'learning_rate': 3.35e-05, 'epoch': 1.0}



 33%|███▎      | 500/1500 [6:20:35<11:33:51, 41.63s/it]

{'eval_loss': 1.8068513870239258, 'eval_runtime': 1317.7651, 'eval_samples_per_second': 0.169, 'eval_steps_per_second': 0.021, 'epoch': 1.0}


 67%|██████▋   | 1000/1500 [12:05:05<5:40:25, 40.85s/it] 

...

Evaluation results: {'eval_loss': 1.7843841314315796, 'eval_runtime': 1284.7724, 'eval_samples_per_second': 0.174, 'eval_steps_per_second': 0.022, 'epoch': 3.0}


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated output: Hello, what is the meaning of life? Is it to be happy, to make others happy, to be successful, to be loved? Is it to be famous, to be rich, to be powerful? Is it to have a family,


C:\Users\panshul2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\panshul2\AppData\Local\Temp\ipykernel_142096\706721376.py:67: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  7%|▋         | 110/1500 [1:21:57<17:40:35, 45.78s/it]

KeyboardInterrupt: 

## Sentiment Analysis

In [4]:
import os
import torch
import zipfile
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
from accelerate import init_empty_weights
from datasets import Dataset

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
 
offload_dir = os.path.expanduser("llama_offload/")
 
os.makedirs(offload_dir, exist_ok=True)

target_file = "task024_cosmosqa_answer_generation.json"
 
# Load tokenizer from Hugging Face
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
tokenizer.pad_token = tokenizer.eos_token
 
# Load the model with accelerate's offloading and device map auto-setup
with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(
        "meta-llama/Meta-Llama-3-8B",
        device_map="auto",
        offload_folder=offload_dir,
        load_in_8bit=True,
        llm_int8_enable_fp32_cpu_offload=True
    )
 
# Configure LoRA with reduced rank
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
C:\Users\panshul2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\accelerate\utils\modeling.py:841: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  _ = torch.tensor([0], device=i)
Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.47s/it]


In [5]:
target_file = r"task1312_amazonreview_polarity_classification.json"
with open(target_file, 'r', encoding='utf-8-sig') as f:
    json_data = json.load(f)


instances = json_data['Instances'][0:2223]
input_texts = [str(instance['input']) for instance in instances]  # Convert to string if not already
output_texts = [str(instance['output'][0]) if instance['output'] else "" for instance in instances]  # Handle missing output
print(input_texts, output_texts)


['I love the brand, my brother has one, and i thought this was great... good brand, well known and the smaller units must not be made to the same standards.', "I love this case! It is sparkly (a true gold, not brassy) and not cheaply made. I love how thin yet durable it is. I've used it everyday since receiving the case and the leopard print has not faded. Worth every penny!", "My husband wore these this week and said they are the best sock he's ever owned! Kept him warm at work where the temps got to freezing and when it was warm during the afternoon, they kept his feet dry and comfy. I'll be buying him more.", 'Handsome looking hooks. Unlike others I managed to get them nearly level and still have the gun level. The finish is mediocre but still acceptable and looks nice as it is. The hooks are padded as well.', 'These are great flip flops but I have to say the sizing can be challenging! I kept going back in forth between an 8.5 and 8 (I’m a true 8.5). Ultimately I went with the 8 bec

In [6]:
 
# Convert the texts into a Hugging Face Dataset
ds = Dataset.from_dict({'input': input_texts, 'output': output_texts})

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["input"], examples["output"], truncation=True, padding="max_length", max_length=512)

# Apply tokenization and set format
tokenized_datasets = ds.map(tokenize_function, batched=True, remove_columns=["input", "output"])
tokenized_datasets.set_format("torch")

# Split dataset into train and eval
train_size = int(0.9 * len(tokenized_datasets))
train_dataset = tokenized_datasets.select(range(train_size))
eval_dataset = tokenized_datasets.select(range(train_size, len(tokenized_datasets)))

# Define data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Map: 100%|██████████| 2223/2223 [00:00<00:00, 6073.96 examples/s]


In [7]:
# Define training arguments
save_dir="finetuned-weights"
training_args = TrainingArguments(
    output_dir=save_dir,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_steps=500,
    save_steps=1000,
    save_total_limit=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    report_to="none",
    fp16=torch.cuda.is_available(),
)
 
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)
 
# Train the model
trainer.train()
 
# Evaluate the model
with torch.no_grad():
    eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")
 
# Save the model and tokenizer

model.save_pretrained(os.path.join(save_dir, "fine-tuned-llama-lora"))
tokenizer.save_pretrained(os.path.join(save_dir, "fine-tuned-llama-lora"))
 
# Test the model on a sample input
input_text = "Hello, what is the meaning of life?"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
 
with torch.no_grad():
    outputs = model.generate(inputs["input_ids"], max_length=50, num_return_sequences=1)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Generated output: {output_text}")
 
# Clear CUDA cache
torch.cuda.empty_cache()

C:\Users\panshul2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\panshul2\AppData\Local\Temp\ipykernel_82172\2425887148.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  0%|          | 0/1500 [00:00<?, ?it/s]C:\Users\panshul2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\models\llama\modeling_llama.py:602: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_

{'loss': 2.9708, 'grad_norm': 2.651029586791992, 'learning_rate': 3.3400000000000005e-05, 'epoch': 1.0}


                                                      
 33%|███▎      | 500/1500 [4:47:57<8:59:27, 32.37s/it]

{'eval_loss': 2.92049503326416, 'eval_runtime': 1214.9482, 'eval_samples_per_second': 0.184, 'eval_steps_per_second': 0.023, 'epoch': 1.0}


 67%|██████▋   | 1000/1500 [9:46:43<5:13:04, 37.57s/it]  

...

{'train_runtime': 55606.7823, 'train_samples_per_second': 0.108, 'train_steps_per_second': 0.027, 'train_loss': 2.7705938313802085, 'epoch': 3.0}


100%|██████████| 28/28 [20:12<00:00, 43.29s/it]


Evaluation results: {'eval_loss': 2.7620482444763184, 'eval_runtime': 1256.2983, 'eval_samples_per_second': 0.178, 'eval_steps_per_second': 0.022, 'epoch': 3.0}


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated output: Hello, what is the meaning of life? Is there a meaning to life? How do you know what you are doing is right? Why is there evil in the world? What is the purpose of life? What is the meaning of life?


In [9]:
input_text = "' I hate going to the movies!' - What is the sentiment of this sentence that i just gave"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
 
with torch.no_grad():
    outputs = model.generate(inputs["input_ids"], max_length=50, num_return_sequences=1)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Generated output: {output_text}")
 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated output: ' I hate going to the movies!' - What is the sentiment of this sentence that i just gave you? It is negative. This is an example of a positive sentence: "I love going to the movies!" This sentence is negative: "
